In [ ]:
%run fungsi_save.ipynb

In [ ]:
#pilih data fitur masukan NN1
train_data = pd.read_csv('data_training.csv', sep=',',header=None)
train_data = train_data.values
post = pd.read_csv('data_feature1_pos.csv', sep=',',header=None)
post = np.int_(post)
total_features = np.size(train_data, axis=1)-2
print(total_features)
output_train_data = train_data[:,total_features+1] #output aja lym
output_train_data = np.int_(output_train_data)
train_data = train_data[:,:total_features]
train_data

In [ ]:
a = [] #index fitur yang gak kepake
for i in range(len(post)):
    if post[i,0] == 0:
        a.append(i)
train_data = np.delete(train_data, a ,axis=1) #data fitur yang udah terseleksi
print("Number of training samples is", len(train_data))
print(np.size(train_data,axis=1))
print(train_data[0])

In [ ]:
#data validasi
validasi_data = pd.read_csv('data_testing.csv', sep=',',header=None)
validasi_data = validasi_data.values
b = [] #index fitur yang gak kepake
for i in range(len(validasi_data)):
    if validasi_data[i,29] == 2:
        b.append(i)
validasi_data = np.delete(validasi_data, b ,axis=0) #data fitur limfo aja
output_validasi_data = validasi_data[:,total_features] #output aja untuk output 2
output_validasi_data = np.int_(output_validasi_data)
validasi_data = validasi_data[:,0:total_features] #fitur aja tanpa output dan fiturnya lengkap
validasi_data = np.delete(validasi_data, a ,axis=1) #data fitur yang udah terseleksi

#save_data1(test_data)

In [ ]:
n_inputs = np.size(train_data,1)
n_hidden = 20
n_classes = 2
n_inputs

In [ ]:
def forward_prop(params):

    # Neural network architecture
    global n_inputs
    global n_hidden
    global n_classes

    w1_total = n_inputs*n_hidden
    b1_total = w1_total+n_hidden
    w2_total = b1_total+(n_hidden*n_classes)
    b2_total = w2_total+n_classes
    
    # Roll-back the weights and biases
    W1 = params[0:w1_total].reshape((n_inputs,n_hidden))
    b1 = params[w1_total:b1_total].reshape((n_hidden,))
    W2 = params[b1_total:w2_total].reshape((n_hidden,n_classes))
    b2 = params[w2_total:b2_total].reshape((n_classes,))

    # Perform forward propagation
    z1 = train_data.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1 #belum pasti pake fungsi aktivasi apa
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood #belum pasti loss functionnya pake apa
    N = len(train_data) # Number of samples
    corect_logprobs = -np.log(probs[range(N), output_train_data])
    loss = np.sum(corect_logprobs) / N
    return loss

In [ ]:
def f(x):
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [ ]:
# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
optimizer = ps.single.GlobalBestPSO(n_particles=20, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=300) #fungsi verbose dan print step error

In [ ]:
from pyswarms.utils.plotters import plot_cost_history
plot_cost_history(cost_history=optimizer.cost_history)
plt.show()

In [ ]:
#kurang test file
def predict(a, pos): #a = data training atau data test

    # Neural network architecture
    global n_inputs
    global n_hidden
    global n_classes

    w1_total = n_inputs*n_hidden
    b1_total = w1_total+n_hidden
    w2_total = b1_total+(n_hidden*n_classes)
    b2_total = w2_total+n_classes
    
    # Roll-back the weights and biases
    W1 = pos[0:w1_total].reshape((n_inputs,n_hidden))
    b1 = pos[w1_total:b1_total].reshape((n_hidden,))
    W2 = pos[b1_total:w2_total].reshape((n_hidden,n_classes))
    b2 = pos[w2_total:b2_total].reshape((n_classes,))
    save_params2(W1,b1,W2,b2)
    # Perform forward propagation
    z1 = a.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # fungsi aktivasi belum fix
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [ ]:
#### data testing
data_test =  pd.read_csv('data_testing_NN1.csv', sep=',',header=None)
data_test = data_test.values
len_data_test = np.size(data_test, axis=1)-1
output_test_data = data_test[:,len_data_test] #output aja limfo
output_test_data = np.int_(output_test_data)
data_test = data_test[:,:len_data_test]
data_test = np.delete(data_test, a ,axis=1) #data fitur yang udah terseleksi

In [ ]:
cm_train = confusion_matrix(output_train_data, predict(train_data, pos))
cm_validasi = confusion_matrix(output_validasi_data, predict(validasi_data, pos))
cm_test = confusion_matrix(output_test_data, predict(data_test, pos))
cm_validasi

In [ ]:
class_names = np.array(['non-lymphoblast',"lymphoblast","non-lymphoid"])
np.set_printoptions(precision=2)
cm_t = plot_confusion_matrix(output_train_data, predict(train_data, pos), classes=class_names, normalize=True,
                      title='Normalized confusion matrix')
cm_v = plot_confusion_matrix(output_validasi_data, predict(validasi_data, pos), classes=class_names, normalize=True,
                      title='Normalized confusion matrix')
cm_te = plot_confusion_matrix(output_test_data, predict(data_test, pos), classes=class_names, normalize=True,
                      title='Normalized confusion matrix')
plt.show()

In [ ]:
cm_t.figure.savefig('fix2(balanced)/step2/NN/train1.png')
cm_v.figure.savefig('fix2(balanced)/step2/NN/validasi1.png')
cm_te.figure.savefig('fix2(balanced)/step2/NN/test1.png')

In [ ]:
print("Presisi data training limfoblas : ", precision(1,cm_train))
print("Recall data training limfoblas (sensitivitas) : ", recall(1,cm_train))
print("Presisi rerata data training : ", precision_macro_average(cm_train))
print("Presisi rerata data training : ", recall_macro_average(cm_train))
print("Akurasi data training : ", accuracy(cm_train))
print("F1 Score data training : ", f1_score(precision_macro_average(cm_train),recall_macro_average(cm_train)))

In [ ]:
print("Presisi data validasi limfoblas : ", precision(1,cm_validasi))
print("Recall data validasi limfoblas : ", recall(1,cm_validasi))
print("Presisi rerata data validasi : ", precision_macro_average(cm_validasi))
print("Presisi rerata data validasi : ", recall_macro_average(cm_validasi))
print("Akurasi data validasi : ", accuracy(cm_validasi))
print("F1 Score data validasi : ", f1_score(precision_macro_average(cm_validasi),recall_macro_average(cm_validasi)))

In [ ]:
print("Presisi data test limfoblas : ", precision(1,cm_test))
print("Recall data test limfoblas : ", recall(1,cm_test))
print("Presisi rerata data test : ", precision_macro_average(cm_test))
print("Presisi rerata data test : ", recall_macro_average(cm_test))
print("Akurasi data test : ", accuracy(cm_test))
print("F1 Score data test : ", f1_score(precision_macro_average(cm_test),recall_macro_average(cm_test)))

In [ ]:
pos2 = [i for i in pos]
pos2

In [ ]:
#save_pos_step1('step2/neuron_step2.csv',(n_hidden,optimizer.cost_history,accuracy(cm_train),accuracy(cm_validasi)))
save_pos_step1('fix2(balanced)/step2/NN/perfo_step2.csv',(accuracy(cm_train),accuracy(cm_validasi),accuracy(cm_test),pos2,optimizer.cost_history))